# A script to compare trajectory datetime with TFR times
from an already joined dataset

In [1]:
import geopandas as gpd
import pandas as pd
import datetime as dt

In [2]:
#create geodataframe with file from script´s directory
gdf = gpd.read_file('TFRs_on_Trajectories.geojson')


#out files as GeoJSON within scrip directory
file_traj_noTFR = 'traj_not_in_TFR.geojson'
file_trai_not_in_TFR_time = 'traj_not_in_TFR_time.geojson'


In [3]:
gdf.head(2)

,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,traj_id,start_t,end_t,length,direction,Source,...,NOTAM__,Issue_Date__UTC_,Effective_Date__UTC_,Cancel_Date__UTC_,Expiration_Date__UTC_,NOTAM_Condition_or_LTA_Subject,Radius,Radius_m,Shape_Length,geometry
0,1,1,1,251,a02862_0,2021-08-01T01:19:23,2021-08-01T01:27:32,8685.843986,39.598238,ZOA_2021-08-01,...,1/8869,07/31/2021 0000,07/31/2021 0015,08/03/2021 1350,10/01/2021 0015,"!FDC 1/8869 ZOA CA..AIRSPACE 24NM E OF CHICO, ...",0.0,0.0,0.091049,"LINESTRING (-121.10220 39.91300, -121.10920 39..."
1,2,1,1,255,a02862_0,2021-08-01T01:19:23,2021-08-01T01:27:32,8685.843986,39.598238,ZOA_2021-08-03,...,1/0014,08/03/2021 0201,08/03/2021 1400,08/04/2021 0359,09/03/2021 0400,"!FDC 1/0014 ZOA CA..AIRSPACE 24NM E OF CHICO, ...",0.0,0.0,0.091049,"LINESTRING (-121.10220 39.91300, -121.10920 39..."


Those aircraft movements having no TFR cover are sorted out

In [4]:
# this cannot have duplicates
gdf_noTFR = gdf[gdf["Join_Count"]==0]

# this can have duplicates in case of multiple TFRs being touched 
# simultaneously or multiple TFRs being issued in the same place
gdf_hasTFR = gdf[gdf["Join_Count"]!=0] 

In [5]:
gdf_noTFR.head(2)

,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,traj_id,start_t,end_t,length,direction,Source,...,NOTAM__,Issue_Date__UTC_,Effective_Date__UTC_,Cancel_Date__UTC_,Expiration_Date__UTC_,NOTAM_Condition_or_LTA_Subject,Radius,Radius_m,Shape_Length,geometry
889,890,0,71,-1,a0956b_10,2021-09-02T22:58:49,2021-09-02T23:02:42,14191.218388,43.931943,None,...,None,None,None,None,None,None,NaN,NaN,0.147856,"LINESTRING (-116.78530 48.28520, -116.76590 48..."
934,935,0,74,-1,a0956b_13,2021-09-22T22:06:19,2021-09-22T22:14:54,45260.857727,282.981352,None,...,None,None,None,None,None,None,NaN,NaN,0.471762,"LINESTRING (-122.32240 38.31120, -122.33190 38..."


This many aircraft movements are not covered by TFRs:

In [6]:
# row count of gdf_noTFR
print(len(gdf_noTFR.index))

86


In [7]:
# just a check: Count non-duplicates (should be 555 here to end up at 555+86 = 641 aircraft movements)
(~gdf_hasTFR.duplicated(["traj_id"])).sum()

555

A trajectories start_t must be within a TFR´s effective time to be covered for sure. That needs calculation per row

In [8]:
gdf_gap = gdf_hasTFR

In [9]:
#get matching TFR
#time formats are all naive but all are known for being UTC
def get_tfrgap(row):
    
    tfrgap = 'not calculated'
    
    enddate = row["Cancel_Date__UTC_"]
    if enddate == None:
        enddate = row["Expiration_Date__UTC_"]
        #if both enddate columns become read, a SettingWithCopyWarning occurs, which is ok
    startdate = row["Effective_Date__UTC_"]
    
    traj_startdate= row["start_t"]
    traj_enddate= row["end_t"]  
    
    #convert to datetime to calculate
    enddate_dt = pd.to_datetime(enddate)
    startdate_dt = pd.to_datetime(startdate)
    traj_startdate_dt = pd.to_datetime(traj_startdate)
    traj_enddate_dt = pd.to_datetime(traj_enddate)
    
    # if trajectory time was within TFR time... 
    if startdate_dt <= traj_startdate_dt and enddate_dt >= traj_enddate_dt:  
        
        #calculate a time gap in minutes
        #this way, a negative value describes TFR being OK, the only case considered here:  
        calcdate= startdate_dt - traj_startdate_dt
        minutes = calcdate.total_seconds()/60
        minutes = int(minutes)
        # need sortable type compared to str 'not calculated':
        minutes = str(minutes)
        tfrgap = minutes
    
    return tfrgap

gdf_gap["TFRgap"]=gdf_hasTFR.apply(get_tfrgap,axis=1)

D:\anaconda3\envs\master_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [10]:
gdf_gap.head(2)

,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,traj_id,start_t,end_t,length,direction,Source,...,Issue_Date__UTC_,Effective_Date__UTC_,Cancel_Date__UTC_,Expiration_Date__UTC_,NOTAM_Condition_or_LTA_Subject,Radius,Radius_m,Shape_Length,geometry,TFRgap
0,1,1,1,251,a02862_0,2021-08-01T01:19:23,2021-08-01T01:27:32,8685.843986,39.598238,ZOA_2021-08-01,...,07/31/2021 0000,07/31/2021 0015,08/03/2021 1350,10/01/2021 0015,"!FDC 1/8869 ZOA CA..AIRSPACE 24NM E OF CHICO, ...",0.0,0.0,0.091049,"LINESTRING (-121.10220 39.91300, -121.10920 39...",-1504
1,2,1,1,255,a02862_0,2021-08-01T01:19:23,2021-08-01T01:27:32,8685.843986,39.598238,ZOA_2021-08-03,...,08/03/2021 0201,08/03/2021 1400,08/04/2021 0359,09/03/2021 0400,"!FDC 1/0014 ZOA CA..AIRSPACE 24NM E OF CHICO, ...",0.0,0.0,0.091049,"LINESTRING (-121.10220 39.91300, -121.10920 39...",not calculated


With the time gap available where possible now, those trajectories (trai_id) need to get removed, where another occurrence of it has a value. Not till then the duplicates can get removed to count trajectories and thus aircraft movements that were not under TFR coverage.

In [11]:
# change the global options that Geopandas inherits from if more rows/columns shall be displayed
#pd.set_option('display.max_rows',None)

In [12]:
#gdf gets sorted and 
#duplicates must be dropped immediately before splitting the sorted gdf
gdf_gap_sorted = gdf_gap.sort_values(["TFRgap"], ascending = True).drop_duplicates(["traj_id"])

#should be 555 here to end up at 555+86 = 641 aircraft movements, 
#unique again but tagged with a minute value for TFRs being OK 
print(len(gdf_gap_sorted.index))

555


In [13]:
#splitting the sorted gdf
gdf_gap_sorted_hascalc = gdf_gap_sorted[gdf_gap_sorted["TFRgap"]!='not calculated'] 
gdf_gap_sorted_nocalc = gdf_gap_sorted[gdf_gap_sorted["TFRgap"]=='not calculated'] 

In [14]:
# Trajectories within TFR timeframe
print('Trajectories within TFR timeframe: Count is '+str(len(gdf_gap_sorted_hascalc.index)))

Trajectories within TFR timeframe: Count is 439


In [15]:
# Trajectories out of TFR timeframe
print('Trajectories out of TFR timeframe: Count is '+str(len(gdf_gap_sorted_nocalc.index)))


Trajectories out of TFR timeframe: Count is 116


In [16]:
#create geojson of questionable trajectories
 # Trajectories outside of any TFR
gdf_noTFR.to_file(filename = file_traj_noTFR, driver='GeoJSON')
 # Trajectories out of TFR timeframe
gdf_gap_sorted_nocalc.to_file(filename = file_trai_not_in_TFR_time, driver='GeoJSON')

D:\anaconda3\envs\master_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
D:\anaconda3\envs\master_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


By how much TFRs for trajectories out of TFR timeframe were late cannot be said. From the gdf_gap_sorted containing duplicates it is not (always) known, which TFR was intended for that particular flight. So the script ends here.